In [6]:
import pandas as pd
from ex_utils.share import ex_path
import json
from pprint import pprint

df = pd.read_csv(
    # ex_path.joinpath("data/thesis2023-fuga ノードリンク図の評価実験（回答） - フォームの回答 1.csv")
    # ex_path.joinpath("data/thesis2023-fuga ノードリンク図の評価実験（回答） - フォームの回答 1 (2).csv")
    ex_path.joinpath(
        "data/thesis2023-fuga ノードリンク図の評価実験（回答） - フォームの回答 1 (3).csv"
    )
)
with open(ex_path.joinpath("data/ex2/pictures.json")) as f:
    data = json.load(f)

remove = ["タイムスタンプ", "氏名", "ノードリンク図の評価実験を通してフィードバックがあれば記入してください。"]
df = df.drop(columns=remove)

df = df.rename(
    columns=dict(
        zip(
            df.columns,
            [c.replace(" [どちらが好ましいですか？]", "") for c in df.columns],
        )
    )
)
# res = res.T.reset_index().rename(columns={"index": "uuid"})

results = df


d_names = sorted(["1138-bus", "USpowerGrid", "qh882", "dwt-1005", "poli"])
target_dataset = d_names[0]
types = ["empirical", "multi", "single"]

score_from_a = {
    "Aが好ましい": 2,
    "Aが少し好ましい": 1,
    "変わらない": 0,
    "Bが少し好ましい": -1,
    "Bが好ましい": -2,
}
N = len(results)
t = len(types)

In [2]:
r_list = []
for i, row in results.iterrows():
    d = row.to_dict()
    results_map = dict(
        zip(
            types,
            [
                dict(zip(types, [0, 0, 0])),
                dict(zip(types, [0, 0, 0])),
                dict(zip(types, [0, 0, 0])),
            ],
        )
    )

    for uuid in d:
        ans = d[uuid]
        target_data = data[uuid]
        a = target_data["A"]
        b = target_data["B"]
        dataset = target_data["dataset"]

        score_a = score_from_a[ans]
        score_b = -score_from_a[ans]

        if dataset == target_dataset:
            results_map[a][b] += score_a
            results_map[b][a] += score_b
    r_list.append(results_map)

n_seed = 3
n_compare = 2
for r in r_list:
    for a in types:
        for b in types:
            r[a][b] /= n_seed * n_compare

In [3]:
r_list[0]

{'empirical': {'empirical': 0.0, 'multi': 1.0, 'single': 0.8333333333333334},
 'multi': {'empirical': -1.0, 'multi': 0.0, 'single': 0.6666666666666666},
 'single': {'empirical': -0.8333333333333334,
  'multi': -0.6666666666666666,
  'single': 0.0}}

In [ ]:
X_i = {}
for type in types:
    X_i[type] = 0


a_means = {}
for type in types:
    a_means[type] = 0

In [ ]:
pprint(r_list[0]["1138-bus"])

In [ ]:
# d_names = sorted(["1138-bus", "USpowerGrid", "qh882", "dwt-1005", "poli"])
# types = ["empirical", "multi", "single"]

# result = dict(
#     zip(
#         d_names,
#         [
#             dict(
#                 zip(
#                     types,
#                     [
#                         dict(zip(types, [0, 0, 0])),
#                         dict(zip(types, [0, 0, 0])),
#                         dict(zip(types, [0, 0, 0])),
#                     ],
#                 )
#             )
#             for _ in range(len(d_names))
#         ],
#     )
# )

# result["all"] = {
#     "empirical": {"empirical": 0, "multi": 0, "single": 0},
#     "multi": {"empirical": 0, "multi": 0, "single": 0},
#     "single": {"empirical": 0, "multi": 0, "single": 0},
# }
# score = dict(
#     zip(
#         d_names,
#         [
#             dict(zip(types, [0, 0, 0])),
#             dict(zip(types, [0, 0, 0])),
#             dict(zip(types, [0, 0, 0])),
#             dict(zip(types, [0, 0, 0])),
#             dict(zip(types, [0, 0, 0])),
#         ],
#     )
# )
# score_all = dict(zip(types, [0, 0, 0]))


# score_from_a = {
#     "Aが好ましい": 2,
#     "Aが少し好ましい": 1,
#     "変わらない": 0,
#     "Bが少し好ましい": -1,
#     "Bが好ましい": -2,
# }
# n = len(res) * 3
# for i, row in res.iterrows():
#     d = row.to_dict()
#     for uuid in d:
#         ans = d[uuid]
#         target_data = data[uuid]
#         a = target_data["A"]
#         b = target_data["B"]
#         dataset = target_data["dataset"]

#         score_a = score_from_a[ans]
#         score_b = -score_from_a[ans]
#         result[dataset][a][b] += score_a
#         result[dataset][b][a] += score_b
#         result["all"][a][b] += score_a
#         result["all"][b][a] += score_b
#         score[dataset][a] += score_a
#         score[dataset][b] += score_b
#         score_all[a] += score_a
#         score_all[b] += score_b

# for d_name in d_names:
#     for a in types:
#         for b in types:
#             result[d_name][a][b] /= n

# for a in types:
#     for b in types:
#         result["all"][a][b] /= n * 5